In [ ]:
from faster_whisper import WhisperModel # type: ignore
import os
from whisper.utils import get_writer # type: ignore
import http.client
import hashlib
import urllib
import random
import json
import unicodedata
import re

In [ ]:
model_size="tiny"
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
"""
Size	Parameters	English-only model	Multilingual model	Required VRAM	Relative speed
tiny	39 M	    tiny.en	            tiny	            ~1 GB	        ~32x
base	74 M	    base.en	            base	            ~1 GB	        ~16x
small	244 M	    small.en	        small	            ~2 GB	        ~6x
medium	769 M	    medium.en	        medium	            ~5 GB	        ~2x
large	1550 M	    N/A	                large	            ~10 GB	        1x
"""

In [ ]:
dir="./"
file="xxx.mp4"
format="lrc" # lrc, srt
file_name=os.path.splitext(file)[0]
appid = 'xxx'        # 填写你的appid
secretKey = 'xxx'# 填写你的密钥
vad_filter=False
language=None
audio=dir+file

In [ ]:
result = model.transcribe(audio, language=language,vad_filter=vad_filter,vad_parameters=dict(min_silence_duration_ms=500))[0]

In [ ]:
def write(file,result,format):
    if format=="lrc":
        lrc_file=file+".lrc"
        with open(lrc_file,'w',encoding='utf-8') as lrcfile:
            for result in result:
                start=convert_time(result.start)[0]
                end=convert_time(result.end)[0]
                lrcfile.write(f"[{start}]{result.text}\n[{end}]\n")
                print(f"{start}")
    elif format=="srt":
        srt_file=file+".srt"
        with open(srt_file,'w',encoding='utf-8') as srtfile:
            for i,result in enumerate(result):
                start=convert_time(result.start)[1]
                end=convert_time(result.end)[1]
                srtfile.write(f"{i}\n{start} --> {end}\n{result.text}\n\n")
                print(f"{start}")
                
def convert_time(seconds):
    # 将秒数转换为毫秒
    milliseconds = int(seconds * 1000)
    
    hours = milliseconds // 3600000
    minutes = milliseconds // 60000
    seconds = (milliseconds % 60000) // 1000
    millis_1 = milliseconds % 100
    millis_2 = milliseconds % 1000
    
    # 格式化输出
    return f"{minutes:02d}:{seconds:02d}.{millis_1:02d}",f"{hours:02d}:{minutes:02d}:{seconds:02d},{millis_2:03d}"

write(f"{dir}{file_name}",result,format)

In [ ]:
def connect():
    try:
        httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
        return httpClient
    except Exception as e:
        print (e)
        httpClient.close()
        
def translate(text,httpClient=None):
    myurl = '/api/trans/vip/translate'  # 通用翻译API HTTP地址
    fromLang = 'auto'       # 原文语种
    toLang = 'zh'           # 译文语种
    salt = random.randint(32768, 65536)
    sign = appid + text + str(salt) + secretKey
    sign = hashlib.md5(sign.encode()).hexdigest()
    myurl = myurl + '?appid=' + appid + '&q=' + urllib.parse.quote(text) + '&from=' + fromLang + \
        '&to=' + toLang + '&salt=' + str(salt) + '&sign=' + sign
    try:
        httpClient.request('GET', myurl)
        # response是HTTPResponse对象
        response = httpClient.getresponse()
        result_all = response.read().decode("utf-8")
        result = json.loads(result_all).get("trans_result")
        result=result[0].get("dst")
        return result
 
    except Exception as e:
        print (e)
        httpClient.close()

def lrc_tran(lrc_file_path):
    new_lrc_file_path = lrc_file_path.replace('.lrc', ' - 翻译.lrc')
    with open(lrc_file_path, 'r', encoding='utf-8-sig') as lrc_file,\
        open(new_lrc_file_path, 'w', encoding='utf-8-sig') as new_lrc_file:
        lines = lrc_file.readlines()
        httpClient=connect()
        for i in range(0, len(lines), 2):
            time_start=re.match(r'\[(\d{2}:\d{2}\.\d{2})\]',lines[i])
            time_end=re.match(r'\[(\d{2}:\d{2}\.\d{2})\]',lines[i+1])
            text = lines[i][time_start.end():].strip()
            time_start = time_start.group(1)
            time_end = time_end.group(1)
            trans=translate(text,httpClient)
            print(f"[{time_start}]{trans}\n")
            new_lrc_file.write(f"[{time_start}]{text}\n[{time_start}]{trans}\n")
        httpClient.close()

def srt_tran(srt_file_path):
    new_srt_file_path = srt_file_path.replace('.srt', ' - 翻译.srt')
    with open(srt_file_path, 'r', encoding='utf-8-sig') as srt_file,\
        open(new_srt_file_path, 'w', encoding='utf-8-sig') as new_srt_file:
        lines = srt_file.readlines()
        httpClient=connect()
        for i in range(0, len(lines), 4):
            trans=translate(lines[i+2].strip(),httpClient)
            new_srt_file.write(f"{lines[i]}{lines[i+1]}{trans}{lines[i+3]}\n")
            print(f"{lines[i+1].strip().split(' --> ')[0]}  {trans}")
        httpClient.close()

def tran(file_name,format):
    if format=="lrc":
        lrc_tran(file_name+".lrc")
    elif format=="srt":
        srt_tran(file_name+".srt")

tran(f"{dir}{file_name}",format)